## Generating fixtures to test the Equation Of Time functions being used and the Longitude Fitting Functions

### Expected Longitude Output is downloaded directly from https://maps.nrel.gov/pvdaq/ PVDAQ Contributed Sites

### Expected longitude = -76.6636

In [1]:
import os
import pandas as pd
from pathlib import Path
import seaborn as sns
import numpy as np
# PVInsight Code Imports
from solardatatools import DataHandler
from solardatatools.dataio import get_pvdaq_data
path = Path.cwd().parent.parent.parent
os.chdir(path)
from pvsystemprofiler.estimator import ConfigurationEstimator
from pvsystemprofiler.longitude_study import LongitudeStudy

### Load data table from external source

#### For today's example, we're loading data from NREL's PVDAQ API, which is a publically available PV generatation data set.

In [2]:
data_frame = get_pvdaq_data(sysid=1199, year=[2015, 2016, 2017], api_key='DEMO_KEY')

CAUTION: Multiple scan rates detected!=======================] 100.0% ...queries complete in 23.4 seconds       
Scan rates (in seconds): [300, 280]
0 transitions detected.
Suggest splitting data set between:




In [3]:
gmt_offset = -5

### Running the  DataHandler

In [8]:
dh = DataHandler(data_frame[0]).fix_dst()

In [7]:
dh.run_pipeline(power_col='ac_power', fix_shifts=False, correct_tz=False)

AttributeError: 'NoneType' object has no attribute 'run_pipeline'

### Calculate the daily solar noon

In [ ]:
from pvsystemprofiler.algorithms.optimized_sunrise_sunset import get_optimized_sunrise_sunset

In [ ]:
fdm = dh.filled_data_matrix
rdm = None
day_of_year = dh.day_index.dayofyear.to_numpy()
days = dh.daily_flags.no_errors
opt_dict = get_optimized_sunrise_sunset(fdm, rdm)

In [ ]:
sunrise = opt_dict['meas_sr_f']
sunset = opt_dict['meas_ss_f']
sunrise[np.isnan(sunrise)] = 0
sunset[np.isnan(sunset)] = 0
solarnoon = np.nanmean([sunrise, sunset], axis=0)

### Equation of time prosposed by Duffie And Rosa

In [ ]:
path = Path.cwd()
os.chdir(path)
path

In [ ]:
# Exporting input for eot algorithms
np.savetxt('./tests/fixtures/longitude_fitting/eot_input.csv', day_of_year,  delimiter=",")

In [ ]:
from pvsystemprofiler.utilities.equation_of_time import eot_da_rosa, eot_duffie

eot_duffie = eot_duffie(day_of_year)
eot_da_rosa = eot_da_rosa(day_of_year)

In [ ]:
# Exporting output for eot algorithms
np.savetxt('./tests/fixtures/longitude_fitting/eot_duffie_output.csv', eot_duffie,  delimiter=",")
np.savetxt('./tests/fixtures/longitude_fitting/eot_da_rosa_output.csv', eot_da_rosa,  delimiter=",")

### Fit longitude

In [ ]:
# exporting inputs for fit_longitude
# solarnoon
np.savetxt('./tests/fixtures/longitude_fitting/solarnoon.csv', solarnoon,  delimiter=",")
# days
np.savetxt('./tests/fixtures/longitude_fitting/days.csv', days,  delimiter=",")

In [ ]:
from pvsystemprofiler.algorithms.longitude.fitting import fit_longitude

fit_longitude(eot_duffie, solarnoon, days, gmt_offset, loss='l2')